In [3]:
!pip install --upgrade plotly

In [4]:
!mkdir ~/.kaggle
!cp kaggle.json ~/.kaggle/
!chmod 600 ~/.kaggle/kaggle.json
!kaggle datasets download -d arjunprasadsarkhel/2021-olymicsin-tokyo
!unzip 2021-olympics-in-tokyo.zip

cp: cannot stat 'kaggle.json': No such file or directory
chmod: cannot access '/root/.kaggle/kaggle.json': No such file or directory
403 - Forbidden - Permission 'datasets.get' was denied
unzip:  cannot find or open 2021-olympics-in-tokyo.zip, 2021-olympics-in-tokyo.zip.zip or 2021-olympics-in-tokyo.zip.ZIP.


In [5]:
!chmod 600 ~/.kaggle/kaggle.json

chmod: cannot access '/root/.kaggle/kaggle.json': No such file or directory


In [6]:
!pip install kaggle

In [7]:
!kaggle datasets download -d arjunprasadsarkhel/2021-olympics-in-tokyo

Dataset URL: https://www.kaggle.com/datasets/arjunprasadsarkhel/2021-olympics-in-tokyo
License(s): CC-BY-SA-4.0
  0% 0.00/348k [00:00<?, ?B/s]
100% 348k/348k [00:00<00:00, 61.7MB/s]


In [8]:
!unzip /content/2021-olympics-in-tokyo.zip -d /content/2021-olympics-in-tokyo

Archive:  /content/2021-olympics-in-tokyo.zip
  inflating: /content/2021-olympics-in-tokyo/Athletes.xlsx  
  inflating: /content/2021-olympics-in-tokyo/Coaches.xlsx  
  inflating: /content/2021-olympics-in-tokyo/EntriesGender.xlsx  
  inflating: /content/2021-olympics-in-tokyo/Medals.xlsx  
  inflating: /content/2021-olympics-in-tokyo/Teams.xlsx  


In [9]:
!pip install Pyspark

In [10]:
import pyspark
spark = pyspark.sql.SparkSession(pyspark.SparkContext())
spark

In [11]:
!mv /content/2021-olympics-in-tokyo/* /content/

In [12]:
import pandas as pd

def excel_to_csv(name):
  pd.read_excel(name + '.xlsx').to_csv(name + '.csv', sep =',', index = False)

excel_to_csv('Athletes')
excel_to_csv('Coaches')
excel_to_csv('EntriesGender')
excel_to_csv('Medals')
excel_to_csv('Teams')

/usr/local/lib/python3.10/dist-packages/openpyxl/styles/stylesheet.py:237: UserWarning: Workbook contains no default style, apply openpyxl's default
  warn("Workbook contains no default style, apply openpyxl's default")
/usr/local/lib/python3.10/dist-packages/openpyxl/styles/stylesheet.py:237: UserWarning: Workbook contains no default style, apply openpyxl's default
  warn("Workbook contains no default style, apply openpyxl's default")


In [13]:
athletes = pd.read_csv('Athletes.csv')
coaches = pd.read_csv('Coaches.csv')
entriesgender = pd.read_csv('EntriesGender.csv')
medals = pd.read_csv('Medals.csv')
teams = pd.read_csv('Teams.csv')

In [14]:
athletes.head()

,Name,NOC,Discipline
0,AALERUD Katrine,Norway,Cycling Road
1,ABAD Nestor,Spain,Artistic Gymnastics
2,ABAGNALE Giovanni,Italy,Rowing
3,ABALDE Alberto,Spain,Basketball
4,ABALDE Tamara,Spain,Basketball


In [15]:
medals.head()

,Rank,Team/NOC,Gold,Silver,Bronze,Total,Rank by Total
0,1,United States of America,39,41,33,113,1
1,2,People's Republic of China,38,32,18,88,2
2,3,Japan,27,14,17,58,5
3,4,Great Britain,22,21,22,65,4
4,5,ROC,20,28,23,71,3


In [16]:
medals.sort_values(by ='Rank by Total', inplace = True)
medals.head()

,Rank,Team/NOC,Gold,Silver,Bronze,Total,Rank by Total
0,1,United States of America,39,41,33,113,1
1,2,People's Republic of China,38,32,18,88,2
4,5,ROC,20,28,23,71,3
3,4,Great Britain,22,21,22,65,4
2,3,Japan,27,14,17,58,5


In [17]:
import plotly.express as px
fig = px.bar(medals, x = 'Team/NOC', y = ['Gold', 'Silver', 'Bronze'], title = 'Number of medals per country')
fig.show()

In [18]:
coaches.head()

,Name,NOC,Discipline,Event
0,ABDELMAGID Wael,Egypt,Football,NaN
1,ABE Junya,Japan,Volleyball,NaN
2,ABE Katsuhiko,Japan,Basketball,NaN
3,ADAMA Cherif,Côte d'Ivoire,Football,NaN
4,AGEBA Yuya,Japan,Volleyball,NaN


In [19]:
spark.read.csv('Coaches.csv', header = True, inferSchema = True).createOrReplaceTempView('coaches')
spark.sql('SELECT * FROM coaches').show(5)

+---------------+-------------+----------+-----+
|           Name|          NOC|Discipline|Event|
+---------------+-------------+----------+-----+
|ABDELMAGID Wael|        Egypt|  Football| NULL|
|      ABE Junya|        Japan|Volleyball| NULL|
|  ABE Katsuhiko|        Japan|Basketball| NULL|
|   ADAMA Cherif|Côte d'Ivoire|  Football| NULL|
|     AGEBA Yuya|        Japan|Volleyball| NULL|
+---------------+-------------+----------+-----+
only showing top 5 rows



In [20]:
pd_coaches_query = spark.sql("SELECT NOC, COUNT(NOC) AS CoachCount FROM coaches GROUP BY NOC ORDER BY COUNT(NOC) DESC").toPandas()

pd_coaches_query.head(5)


,NOC,CoachCount
0,Japan,35
1,Spain,28
2,United States of America,28
3,Australia,22
4,Canada,16


In [21]:
fig = px.bar(pd_coaches_query, x = 'NOC', y = 'CoachCount', color= 'CoachCount',title = 'Numbe of coaches from each country')
fig.show()

In [22]:
athletes.head()

,Name,NOC,Discipline
0,AALERUD Katrine,Norway,Cycling Road
1,ABAD Nestor,Spain,Artistic Gymnastics
2,ABAGNALE Giovanni,Italy,Rowing
3,ABALDE Alberto,Spain,Basketball
4,ABALDE Tamara,Spain,Basketball


In [23]:
spark.read.csv('Athletes.csv', header = True, inferSchema = True).createOrReplaceTempView('athletes')
spark.sql('SELECT * FROM Athletes ORDER BY NOC, Discipline').show(20)

+--------------------+-----------+-------------------+
|                Name|        NOC|         Discipline|
+--------------------+-----------+-------------------+
|       YOUSOFI Kimia|Afghanistan|          Athletics|
|NOOR ZAHI Sha Mah...|Afghanistan|          Athletics|
|        YOVARI Mahdi|Afghanistan|           Shooting|
|        ANWARI Fahim|Afghanistan|           Swimming|
|     MANSOURI Farzad|Afghanistan|          Taekwondo|
|       PETROV Matvei|    Albania|Artistic Gymnastics|
|       SMAJLAJ Izmir|    Albania|          Athletics|
|      CULLHAJ Indrit|    Albania|               Judo|
|     DELILAJ Manuela|    Albania|           Shooting|
|         KADIU Kledi|    Albania|           Swimming|
|       MERIZAJ Nikol|    Albania|           Swimming|
|        CALJA Briken|    Albania|      Weightlifting|
|      QERIMAJ Erkand|    Albania|      Weightlifting|
|      HETHAT Yassine|    Algeria|          Athletics|
| LAHOULOU Abdelmalik|    Algeria|          Athletics|
|TRIKI Yas

In [24]:
spark.sql('''SELECT FIRST(NOC) AS FirstNOC
              , FIRST(Discipline) AS FirstDiscipline
              , COUNT(*) AS CountOfAthletes
              FROM
                athletes
              GROUP BY
                NOC
                , Discipline
              ORDER BY
                NOC
                , Discipline''').show(5)


+-----------+-------------------+---------------+
|   FirstNOC|    FirstDiscipline|CountOfAthletes|
+-----------+-------------------+---------------+
|Afghanistan|          Athletics|              2|
|Afghanistan|           Shooting|              1|
|Afghanistan|           Swimming|              1|
|Afghanistan|          Taekwondo|              1|
|    Albania|Artistic Gymnastics|              1|
+-----------+-------------------+---------------+
only showing top 5 rows



In [25]:
pd_athletes_query = spark.sql('''SELECT FIRST(NOC) AS FirstNOC
              , FIRST(Discipline) AS FirstDiscipline
              , COUNT(*) AS CountOfAthletes
              FROM
                athletes
              GROUP BY
                NOC
                , Discipline
              ORDER BY
                NOC
                , Discipline''').toPandas()

In [26]:
pd_athletes_query.head(5)

,FirstNOC,FirstDiscipline,CountOfAthletes
0,Afghanistan,Athletics,2
1,Afghanistan,Shooting,1
2,Afghanistan,Swimming,1
3,Afghanistan,Taekwondo,1
4,Albania,Artistic Gymnastics,1


In [27]:
fig = px.bar(pd_athletes_query, x = 'FirstNOC', y = 'CountOfAthletes', color = 'FirstDiscipline', title = 'Number Of Athlestes per country and their respective discipline').show()

In [28]:
athletes['Discipline'].value_counts()

,count
Discipline,
Athletics,2068
Swimming,743
Football,567
Rowing,496
Hockey,406
Judo,373
Handball,343
Shooting,342
Sailing,336


In [29]:
print(pd_athletes_query.columns)

Index(['FirstNOC', 'FirstDiscipline', 'CountOfAthletes'], dtype='object')


In [30]:
pd_athletes_query = pd_athletes_query.loc[pd_athletes_query['FirstDiscipline'].isin(['Athletics', 'Swimming', 'Football'])]

In [31]:
pd_athletes_query.FirstDiscipline.value_counts()

,count
FirstDiscipline,
Athletics,195
Swimming,181
Football,24


In [32]:
fig = px.bar(pd_athletes_query, x = 'FirstNOC', y = 'CountOfAthletes', color = 'FirstDiscipline', title = 'Number Of Athlestes per country and their respective discipline').show()

In [34]:
entriesgender.head(5)

,Discipline,Female,Male,Total
0,3x3 Basketball,32,32,64
1,Archery,64,64,128
2,Artistic Gymnastics,98,98,196
3,Artistic Swimming,105,0,105
4,Athletics,969,1072,2041


In [36]:
len(entriesgender)

46

In [41]:
from plotly.subplots import make_subplots
import plotly.graph_objects as go

specs = [[{'type': 'domain'}, {'type': 'domain'}]] * 23

specs

fig = make_subplots(rows = 23, cols = 2, subplot_titles = entriesgender['Discipline'], specs = specs)
sub_figs = []

for index, row in entriesgender.iterrows():
  sub_fig = go.Pie(labels = ['Female', 'Male'], values = [row['Female']
                                                          , row['Male']])
  sub_figs.append(sub_fig)

k = 0
for i in range(1, 24):
  for j in range(1,3):

    fig.add_trace(sub_figs[k], i, j)
    k += 1

fig.update_layout(showlegend = False, height = 10000, width = 800, title = 'Distribution of gender amongst each game')
fig.update_traces(textposition = 'inside', textinfo = 'percent+label')

fig = go.Figure(fig)
fig.show()


